In [16]:
import os
import exifread
import requests
import json
import datetime

In [2]:
def transform_GPScoordinate(GPSLongitude, GPSLatitude ):
    '''
    把地图上的GPS经纬度信息转换为数值型的GPS坐标
    '''
    ## 经度
    logitude_list = GPSLongitude.replace("[", "").replace("]", "").split(',')
    logitude_list = [element.strip() for element in  logitude_list]
    degree = int(logitude_list[0])  # 度
    minute = int(logitude_list[1]) / 60 # 分
    temp = logitude_list[2].split('/')
    second = int(temp[0]) / int(temp[1]) / 3600
    logitude = degree + minute + second
    
    ## 纬度
    latitude_list = GPSLatitude.replace("[", "").replace("]", "").split(',')
    latitude_list = [element.strip() for element in  latitude_list]
    degree = int(latitude_list[0])  # 度
    minute = int(latitude_list[1]) / 60 # 分
    temp = latitude_list[2].split('/')
    second = int(temp[0]) / int(temp[1]) / 3600
    latitude = degree + minute + second
    
    return "%0.6f" %logitude, "%0.6f" %latitude
    

In [7]:
def GPS_to_gaodeCoordinate(GPS_coordinate):
    '''
    GPS坐标转高德地图坐标
    '''
    logitude, latitude = GPS_coordinate
    URL = 'https://restapi.amap.com/v3/assistant/coordinate/convert?locations={},{}&coordsys=gps&key={}'.format(
        logitude,latitude, key)
    response = requests.get(URL)
    location_json = json.loads(response.text)

    if location_json['status'] is '1':
        coordinate = location_json['locations']
        return coordinate
    else:
        print('转换高德坐标不成功！')

In [8]:
def get_address(coordinate):
    '''
    通过高德坐标请求地理位置信息
    '''
    coordinate = ",".join(['%.06f' % float(coordinate.split(",")[0]), '%.06f' % float(coordinate.split(",")[1]) ])
    URL = "https://restapi.amap.com/v3/geocode/regeo?location={}&key={}".format(coordinate, key )
    response = requests.get(URL)
    address_json = json.loads(response.text)
    if address_json['status'] is  '1':
        location = address_json['regeocode']['formatted_address']
        return location
    else:
        print('请求位置信息失败！')

In [9]:
class Location():
    def __init__(self,img_path):
        img_exif = exifread.process_file(open(img_path, 'rb'))
        self.date_time = str(img_exif['EXIF DateTimeOriginal']) # 拍摄时间
#         self.GPSLatitudeRef = str(img_exif['GPS GPSLatitudeRef']) # 纬度方向
#         self.GPSLongitudeRef = str(img_exif[ 'GPS GPSLongitudeRef']) ## 经度方向
        self.GPSLatitude = str(img_exif['GPS GPSLatitude']) ## 纬度
        self.GPSLongitude = str(img_exif['GPS GPSLongitude']) ## 经度
        
        ## https://lbs.amap.com/dev/key/app 在高德开发者平台申请key
        self.key = "265421ba2620841a9c*********"
        


In [13]:
img_path = "xxxx.jpg"
img_exif = Location(img_path)
key = img_exif.key
## 解析图片的GPS位置坐标
GPS_coordinate = transform_GPScoordinate(img_exif.GPSLongitude, img_exif.GPSLatitude )
## GPS坐标 转换为 高德位置坐标
coordinate = GPS_to_gaodeCoordinate(GPS_coordinate)
## 请求高德地图为位置
location = get_address(coordinate)

